<a href="https://colab.research.google.com/github/adithyaac/Captcha-breaker/blob/main/Task%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>